<a href="https://colab.research.google.com/github/alberttang35/DL-4/blob/main/DeepLearning4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ucimlrepo

In [40]:
import numpy as np
import keras
from keras import layers
from keras import ops
from ucimlrepo import fetch_ucirepo
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.layers import Input, Add, Dense, Embedding, Flatten, ReLU
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [41]:
# fetch dataset
covertype = fetch_ucirepo(id=31)

# data (as pandas dataframes)
X = covertype.data.features
y = covertype.data.targets

# we say there are 8 classes, because the dataset is 1-indexed
nb_classes = 8

# metadata
print(covertype.metadata)

# variable information
print(covertype.variables)

{'uci_id': 31, 'name': 'Covertype', 'repository_url': 'https://archive.ics.uci.edu/dataset/31/covertype', 'data_url': 'https://archive.ics.uci.edu/static/public/31/data.csv', 'abstract': 'Classification of pixels into 7 forest cover types based on attributes such as elevation, aspect, slope, hillshade, soil-type, and more.', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 581012, 'num_features': 54, 'feature_types': ['Categorical', 'Integer'], 'demographics': [], 'target_col': ['Cover_Type'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1998, 'last_updated': 'Sat Mar 16 2024', 'dataset_doi': '10.24432/C50K5N', 'creators': ['Jock Blackard'], 'intro_paper': None, 'additional_info': {'summary': 'Predicting forest cover type from cartographic variables only (no remotely sensed data).  The actual forest cover type for a given observation (30 x 30 meter cell) was determined from

In [77]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.8, random_state=42)

Y_train = to_categorical(y_train, nb_classes)
Y_valid = to_categorical(y_valid, nb_classes)
batch_size = 128

X_train = X_train[:batch_size]
Y_train = Y_train[:batch_size]

In [78]:
inputs = keras.Input(shape=(54,))
dense1 = Dense(32, activation="relu")
dense1_out = dense1(inputs)

dense2 = Dense(32, activation="relu")
dense2_out = dense2(dense1_out)

residual_connection = Add()([dense1_out, dense2_out])

dense3 = Dense(32, activation="relu")
dense3_out = dense3(residual_connection)

skip_connection = Dense(32)(inputs)
skip_addition = Add()([skip_connection, dense3_out])
dense4_out = ReLU()(skip_addition)

dense5 = Dense(32, activation="relu")
dense5_out = dense5(dense4_out)

# dense6 = Dense(32, activation="relu")
# dense6_out = dense6(dense5_out)

outputs = layers.Dense(nb_classes, activation="softmax")(dense5_out)
model = keras.Model(inputs=inputs, outputs=outputs)


In [79]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.001, momentum=0.9), metrics=['accuracy'])

network_history = model.fit(X_train, Y_train, batch_size=128,
                            epochs=30, verbose=1, validation_data=(X_valid, Y_valid), callbacks=[])

Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.0312 - loss: 1530.2521 - val_accuracy: 0.4881 - val_loss: 946.0170
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.5078 - loss: 871.2670 - val_accuracy: 0.3639 - val_loss: 84313.6094
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - accuracy: 0.3438 - loss: 86071.6641 - val_accuracy: 0.4881 - val_loss: 52800.2305
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 0.5078 - loss: 49880.0938 - val_accuracy: 0.3639 - val_loss: 2835313.5000
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step - accuracy: 0.3438 - loss: 2637223.0000 - val_accuracy: 0.4881 - val_loss: 3.1729
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - accuracy: 0.5078 - loss: 2.0731 - val_accuracy: 0.4881 - val_loss: 4.1824
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step - accuracy: 0.5078 - loss: 2.0790 - val_accuracy: 0.4881 - val_loss: 5.1262
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step - accuracy: 0.5078 - loss: 2.1179 - val_acc

In [80]:
model.summary()

Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20            │ (None, 54)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_110 (Dense)         │ (None, 32)             │          1,760 │ input_layer_20[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_111 (Dense)         │ (None, 32)             │          1,056 │ dense_110[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_38 (Add)              │ (None, 32)             │              0 │ dense_110[0][0],       │
│                           │                        │                │ dense_111[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_113 (Dense)         │ (None, 32)             │          1,760 │ input_layer_20[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_112 (Dense)         │ (None, 32)             │          1,056 │ add_38[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_39 (Add)              │ (None, 32)             │              0 │ dense_113[0][0],       │
│                           │                        │                │ dense_112[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_18 (ReLU)           │ (None, 32)             │              0 │ add_39[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_114 (Dense)         │ (None, 32)             │          1,056 │ re_lu_18[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_115 (Dense)         │ (None, 8)              │            264 │ dense_114[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 13,906 (54.32 KB)

 Trainable params: 6,952 (27.16 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,954 (27.17 KB)

In [81]:
# final train loss
print(network_history.history["loss"][-1])
# final validation loss
print(network_history.history["val_loss"][-1])

2.0392913818359375
12.223068237304688


In [82]:
model.save('DL_4_model.h5')